<a href="https://colab.research.google.com/github/Dkepffl/2022-2-ESAA/blob/main/Assignment/Assignment220923_Ensemble.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# import package
import numpy as np
import os

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

In [2]:
np.random.seed(42)

# **CHAPTER 7 앙상블 학습과 랜덤 포레스트**
___
- 일련의 예측 모델(즉, 분류나 회귀 모델)로부터 예측을 수집하면 가장 좋은 모델 하나보다 더 좋은 예측을 얻을 수 있을 수 있다. 이 일련의 예측기를 **앙상블**이라고 하며, 이 모델을 학습 시키는 것을 **앙상블 학습(ensemble learning)**이라고 부른다. 또한 앙상블 학습 알고리즘을 **앙상블 방법(ensemble method)**라고 한다.
- 앙상블 방법 중 한 가지로 훈련 세트로부터 무작위로 각기 다른 서브셋을 만들어 일련의 결정 트리 분류기를 훈련시킨 다음, 모든 개별 트리의 예측을 구해 가장 많은 선택을 받은 클래스를 예측으로 삼는 결정 트리의 앙상블 **랜덤 포레스트(random forest)**라고 한다.
- 이 장에서는 **배깅, 부스팅, 스태킹** 등 가장 인기 있는 앙상블 방법을 설명한다.

## **7.1 투표 기반 분류기**
____
- 정확도가 80%인 분류 모델 여러 개(예 : 로지스틱 회귀 분류 모델, SVM 분류 모델, 랜덤 포레스트 분류 모델, KNN 분류 모델 등)를 훈련시켰다고 가정하자. 
- 더 좋은 분류 모델을 만드는 매우 간단한 방법은 각 분류 모델의 예측을 모아, 가장 많이 선택된 클래스를 예측하는 것이다. 이렇게 다수결 투표로 정해지는 분류 모델을 **직접 투표(hand voting)** 분류 모델이라고 한다.
  - 이 다수결 투표 분류 모델이 앙상블에 포함된 개별 분류 모델 중 가장 뛰어난 것보다 정확도가 높을 수 있다. 
  - 또한 각 분류 모델이 **약한 학습기(weak learner, 랜덤 추측보다 조금 더 높은 성능을 내는 분류 모델)**이라도 충분히 많고 다양하다면 앙상블은 **강한 학습기(strong learner, 높은 정확도를 가진 분류 모델)**라고 한다.
- 앙상블 모델은 각 분류 모델이 독립적이고 오차에 상관 관계가 적을수록 좋은 성능을 낸다. 
  + 같은 데이터로 훈련시키기 때문에 완벽하게 독립적이고 오차에 상관 관계가 없을 수는 없다.
  + 따라서 각기 다른 알고리즘으로 학습시켜야 매우 다른 종류의 오차를 만들 가능성이 높아, 앙상블 모델의 정확도를 향상시킨다.
- 다음은 여러 분류 모델을 조합하여 사이킷런의 투표 기반 분류기(`VotingClassifier`)를 만들고 훈련시키는 코드이다.

In [3]:
# import data
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_moons

X, y = make_moons(n_samples=500, noise=0.30, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [4]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

# 각 분류 모델 학습
log_clf = LogisticRegression(solver="lbfgs", random_state=42)
rnd_clf = RandomForestClassifier(n_estimators=100, random_state=42)
svm_clf = SVC(gamma="scale", random_state=42)

In [5]:
# 투표 기반 분류기 생성
voting_clf = VotingClassifier(estimators=[('lr', log_clf), ('rf', rnd_clf), ('svc', svm_clf)], 
                              voting='hard')

# 분류 모델 학습
voting_clf.fit(X_train, y_train)

VotingClassifier(estimators=[('lr', LogisticRegression(random_state=42)),
                             ('rf', RandomForestClassifier(random_state=42)),
                             ('svc', SVC(random_state=42))])

- 각 분류 모델과 투표 기반 분로 모델의 테스트셋에서의 정확도를 확인해보자.

In [6]:
from sklearn.metrics import accuracy_score

for clf in (log_clf, rnd_clf, svm_clf, voting_clf):
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print(clf.__class__.__name__, accuracy_score(y_test, y_pred))

LogisticRegression 0.864
RandomForestClassifier 0.896
SVC 0.896
VotingClassifier 0.912


- 투표 기반 분류 모델이 다른 개별 분류 모델보다 성능이 조금 더 높다.
- 모든 분류 모델가 클래스의 확률을 예측할 수 있으면(즉, `predict_proba()` 메서드가 있으면) 개별 분류 모델의 예측을 평균 내어 확률이 가장 높은 클래스를 예측할 수 있다. 이를 **간접 투표(soft voting)**라고 한다.
  + 확률이 높은 투표에 비중을 더 두어 직접 투표 방식보다 성능이 높다.
  + 사이킷런의 `VotingClassifier()` 클래스에서 이 방식을 사용하려면, `voting='hard'`를 `voting='soft'`로 바꾸고 모든 분류 모델의 클래스 확률을 추정할 수 있으면 된다.
  + `SVC`는 기본값에서는 클래스 확률을 제공하지 않으므로, `probability` 매개변수를 `True`로 지정해야 한다.

## **7.2 배깅과 페이스팅**
___
- 앞서 말했듯이 다양한 분류기를 만드는 한 가지 방법은 각기 다른 훈련 알고리즘을 사용하는 것이다. 또 다른 방법은 같은 알고리즘을 사용하고 훈련 세트의 서브셋을 무작위로 구성하여 분류기를 각기 다르게 학습시키는 것이다.
  + 훈련 세트에서 중복을 허용하여 샘플링하는 방식을 **배깅(bagging)**이라고 한다.
  + 중복을 허용하지 않고 샘플링하는 방식을 **페이스팅(pasting)**이라고 한다.
- 모든 예측 모델이 훈련을 마치면, 앙상블은 모든 예측 모델의 예측을 모아서 새로운 샘플에 대한 예측을 만든다. 수집 함수는 **분류일 때 보통 통계적 최빈값**(직접 투표 분류 모델처럼 가장 많은 예측 결과)이고 **회귀에 대해서는 평균**을 계산한다.
  + 개별 예측 모델은 원본 훈련 세트로 훈련시킨 것보다 훨씬 크게 편향되어 있지만 수집 함수를 통과하면 편향과 분산이 모두 감소한다.
  + 일반적으로 앙상블의 결과는 원본 데이터셋으로 하나의 예측기를 훈련시킬 때와 비교해 편향과 비슷하지만 분산은 줄어든다.

#### | **7.2.1 사이킷런의 배깅과 페이스팅**
- 사이킷런은 배깅과 페이스팅을 위해 간편한 API로 구성된 `BaggingClassifier`(회귀의 경우, `BaggingRegressor`)를 제공한다.
- 다음은 결정 트리 분류 모델 500개의 앙상블을 훈련시키는 코드이다.
  + 각 분류 모델은 훈련 세트에서 중복을 허용하여 무작위로 선택된 100개의 샘플로 훈련된다. 즉, 배깅 방식이다. 페이스팅을 사용하려면 `bootstrap=False`로 설정하면 도니다.
  + `n_jobs` 매개변수는 사이킷런이 훈련과 예측에 사용할 CPU 코어 수를 결정한다.


In [7]:
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier

bag_clf = BaggingClassifier(DecisionTreeClassifier(), n_estimators=500,
    max_samples=100, bootstrap=True, random_state=42)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

- 부트스트래핑은 각 예측기가 학습하는 서브셋에 다양성을 증가시키므로, 배깅이 페이스팅보다 편향이 조금 더 높다.
- 하지만 다양성을 추가한다는 것은 예측 모델 사이의 상관 관계를 줄이므로, 앙상블의 분산을 감소시킨다.
- 전반적으로 배깅이 더 나은 모델을 만들기 때문에 일반적으로 배깅을 더 선호한다. 하지만 시간과 CPU 파워에 여유가 있다면 교차 검증으로 배깅과 페이스팅을 모두 평가해 더 나은 쪽을 선택하는 것이 좋다.

#### | **7.2.2 oob 평가**
- 배깅을 사용하면 어떤 샘플은 한 예측 모델을 위해 여러 번 샘플링 되고, 어떤 샘플은 전혀 선택되지 않을 수 있다.
- `BaggingClassifier`는 기본적으로 중복을 허용하여(Default : `boootstrap=True`) 훈련 세트의 크기(=m) 만큼 샘플을 선택한다. 이는 평균적으로 각 예측 모델에 훈련 샘플의 63% 정도만 샘플링된다는 것을 의미한다. 
  + 이때 선택되지 않은 훈련 샘플의 나머지 37%를 oob 샘플(out-of-bag)이라고 부른다.
  + 예측마다 남겨진 oob 샘플은 모두 다르다.
- 예측 모델이 훈련되는 동안 oob 샘플을 사용하지 않으므로, 별도의 검증 세트를 사용하지 않고 oob 샘플을 사용해 평가할 수 있다. 앙상블 모델의 평가는 각 예측 모델의 oob 평가를 평균하여 얻는다.
- 사이킷런에서 `BaggingClassifier`를 만들 때, `oob_score=True`라고 설정하면, 훈련이 끝난 후 자동으로 oob 평가를 수행하고 결과를 `oob_score_` 변수에 저장한다.


In [10]:
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(), n_estimators=500,
    bootstrap=True, oob_score=True, random_state=40)
bag_clf.fit(X_train, y_train)

bag_clf.oob_score_

0.8986666666666666

- oob 평가 결과를 보면 이 `BaggingClassifier`는 테스트셋에서 약 89.9%의 정확도를 얻을 것으로 보인다.

In [11]:
# 실제 테스트셋에서의 정확도
from sklearn.metrics import accuracy_score

y_pred = bag_clf.predict(X_test)
accuracy_score(y_test, y_pred)

0.912

- 테스트셋에서 91.2%의 정확도를 얻었다.
- oob 샘플에 대한 결정 함수의 값도 `oob_decision_function` 변수에서 확인할 수 있다. 이 경우 기반이 되는 예측 모델이 `predict_proba()` 메서드를 가지고 있어 결정 함수는 각 훈련 샘플의 클래스 확률을 반환한다.

In [12]:
bag_clf.oob_decision_function_

array([[0.32275132, 0.67724868],
       [0.34117647, 0.65882353],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.09497207, 0.90502793],
       [0.31147541, 0.68852459],
       [0.01754386, 0.98245614],
       [0.97109827, 0.02890173],
       [0.97765363, 0.02234637],
       [0.74404762, 0.25595238],
       [0.        , 1.        ],
       [0.7173913 , 0.2826087 ],
       [0.85026738, 0.14973262],
       [0.97222222, 0.02777778],
       [0.0625    , 0.9375    ],
       [0.        , 1.        ],
       [0.97837838, 0.02162162],
       [0.94642857, 0.05357143],
       [1.        , 0.        ],
       [0.01704545, 0.98295455],
       [0.39473684, 0.60526316],
       [0.88700565, 0.11299435],
       [1.        , 0.        ],
       [0.97790055, 0.02209945],
       [0.        , 1.        ],
       [0.99428571, 0.00571429],
       [1.        , 0.        ],
       [0.        , 1.        ],
       [0.62569832, 0.37430168],
       [0.

## **7.3 랜덤 패치와 랜덤 서브스페이스**
____
- `BaggingClassifier`는 피처 샘플링도 지원한다. 샘플링은 `max_features`, `bootstrap_features` 두 매개변수로 조절된다.
  + 작동 방식은 `max_samples`, `bootstrap`과 동일하지만 샘플이 아니고 피처에 대한 샘플링이다.
  + 각 예측 모델은 무작위로 선택한 입력 피처의 일부분으로 훈련된다.
  + 이 기법은 이미지와 같은 매우 고차원의 데이터셋을 다룰 때 유용하다.
- 훈련 피처와 샘플을 모두 샘플링하는 것을 **랜덤 패치 방식(random patches method)**이라고 한다.
- 훈련 샘플을 모두 사용하고 피처는 샘플링하는 것은 **랜덤 서브스페이스 방식(random subspaces method)**이라고 한다.
- 피처 샘플링은 더 다양한 예측 모델을 만들어 편향을 늘리는 대신 분산을 줄인다.


## **7.4 랜덤 포레스트**
____
- 랜덤 포레스트는 일반적으로 배깅 방법(또는 페이스팅)을 적용한 결정 트리의 앙상블이다.
  + `max_samples`로 훈련 세트의 크기를 지정한다.
  + 결정 트리에 최적화되어 사용하기 편리한 `RandomForestClassifier`를 사용할 수 있다.
  + 회귀 문제를 위해서는 `RandomForestRegressor`를 사용한다.

In [13]:
from sklearn.ensemble import RandomForestClassifier

# 500개의 트리로 이루어진 랜덤 포레스트 모델
rnd_clf = RandomForestClassifier(n_estimators=500, max_leaf_nodes=16, random_state=42)
rnd_clf.fit(X_train, y_train)

y_pred_rf = rnd_clf.predict(X_test)

- 랜덤 포레스트 알고리즘은 트리의 노드를 분할할 때 전체 피처 중에서 최선의 피처를 찾는 대신 무작위로 선택한 피처 후부 중에서 최적의 피처를 찾는 식으로 무작위성을 더 주입한다.
- 이는 결국 트리를 더욱 다양하게 만들고 편향을 늘리는 대신 분산을 낮추어 전체적으로 더 훌륭한 모델을 만들어낸다.
- 다음은 `BaggingClassifier`를 사용해 앞의 `RandomForestClassifier`와 거의 유사하게 만든 것이다.

In [14]:
# BaggingClassifier를 사용해 만든 랜덤 포레스트 모델
bag_clf = BaggingClassifier(
    DecisionTreeClassifier(max_features="sqrt", max_leaf_nodes=16),
    n_estimators=500, random_state=42)

bag_clf.fit(X_train, y_train)
y_pred = bag_clf.predict(X_test)

In [15]:
# 예측 값 비교
np.sum(y_pred == y_pred_rf) / len(y_pred)

1.0

#### | **7.4.1 엑스트라 트리**
- 랜덤 포레스트에서 트리를 만들 때 각 노드는 무작위로 피처의 서브셋을 만들어 분할에 사용한다.
- 트리를 더욱 무작위하게 만들기 위해 최적의 임계값을 찾는 대신 후보 피처를 사용해 무작위로 분할한 다음 그중에서 최상의 분할을 선택한다.
- 이와 같이 극단적으로 무작위한 트리의 랜덤 포레스트를 **익스트림 랜덤 트리(extremely randomized trees)** 앙상불 혹은 줄여서 엑스트라 트리라고 부른다.
  + 여기서도 편향이 늘어나지만 분산이 줄어든다.
  + 모든 노드에서 피처마다 가장 최적의 임계값을 찾아 트리 알고리즘 시간이 길어지는데, 엑스트라 트리는 그렇지 않기 때문에 훨씬 빠르다.
- 엑스트라 트리는 사이킷런의 `ExtraTreesClassifier`를 사용한다.

#### | **7.4.2 특성 중요도**
- 랜덤 포레스트의 또 다른 장점은 피처의 상대적 중요도를 측정하기 쉽다는 것이다. 
  + 사이킷런은 어떤 피처를 사용한 노드가 평균적으로 불순도를 얼마나 감소시키는지 계산하여 피처의 중요도를 측정한다.
  + 정확히는 가중치 평균이며, 각 노드의 가중치는 연관된 훈련 샘플 수와 같다.
- 사이킷런은 훈련이 끝난 뒤 자동으로 이 점수를 계산하고 중요도의 전체 합이 1이 되도록 결과값을 정규화한다.
  + 이 값은 `feature_importances_` 변수에 저장되어 있다.

In [16]:
from sklearn.datasets import load_iris

# import data
iris = load_iris()

In [17]:
# model fitting
rnd_clf = RandomForestClassifier(n_estimators=500, random_state=42)
rnd_clf.fit(iris["data"], iris["target"])

RandomForestClassifier(n_estimators=500, random_state=42)

In [18]:
# print the importances of features
for name, score in zip(iris["feature_names"], rnd_clf.feature_importances_):
    print(name, score)

sepal length (cm) 0.11249225099876375
sepal width (cm) 0.02311928828251033
petal length (cm) 0.4410304643639577
petal width (cm) 0.4233579963547682


## **7.5 부스팅**
____
- 부스팅(boosting)은 약한 학습기를 여러 개 연결하여 강한 학습기를 만드는 앙상블 방법을 말한다. 이때 앞의 모델을 보완하면서 일련의 예측 모델을 학습시킨다.
- 부스팅 방법에는 여러 가지가 있지만 가장 인기있는 것은 에이다부스트와 그레디언트 부스팅이다.

#### | **7.5.1 에이다부스트**
- 이전 예측기를 보완하는 새로운 예측기를 만드는 방법은 이전 모델이 과소적합했던 훈련 샘플의 가중치를 더 높이는 것이다. 이렇게 하면 새로운 예측기는 학습하기 어려운 샘플에 점점 더 맞춰지게 된다. 이것이 **에이다부스트**에서 사용하는 방식이다.
  + 분류 모델을 훈련 세트에서 훈련시키고 예측을 만든 다음 잘못 분류된 훈련 샘플의 가중치를 상대적으로 높이는 것을 반복한다.
- 모든 예측 모델의 훈련이 끝나면, 이 앙상블 모델은 배깅이나 페이스팅과 비슷한 방식으로 예측을 만든다. 하지만 가중치가 적용된 훈련 세트의 전반적인 정확도에 따라 예측 모델마다 다른 가중치가 적용된다.
- 에이다부스트 훈련 알고리즘
  1. 각 샘플에 대한 가중치 $w^{(i)}$를 $\frac{1}{m}$으로 초기화하고, 첫 번째 모델을 학습한다.
  2. 가중치가 적용된 에러율 $r_1$을 훈련 세트에 대해 계산한다.
  $$r_j = \frac{\sum_{i=1, \hat{y_j} \ne y^{(i)}}^{m}w^{(i)}}{\sum_{i=1}^{m} w^{(i)}}$$
  3. 그리고 예측 모델의 가중치 $\alpha_j$를 계산한다. 여기서 $\eta$는 학습률 하이퍼 파라미터이다. 예측 모델이 정확할수록 가중치가 더 높아진다. 
  $$\alpha_j= \eta log{\frac{1-r_j}{r_j}}$$
  4. 아래 식을 이용하여 샘플의 가중치를 업데이트한다. 즉, 잘못 분류된 샘플의 가중치가 증가한다.$i = 1, 2, 3, \cdots$
  $$ w^{(i)} = 
  \begin{cases}
  w^{(i)}, & \mbox{if }\hat{y_j} = y^{(i)} \\
  w^{(i)} exp(\alpha_j), & \mbox{if }\hat{y_j} \ne y^{(i)}
  \end{cases} $$
  5. 그리고 모든 샘플의 가중치를 정규화한다.
  6. 마지막으로 새 예측 모델이 업데이트된 가중치를 사용해 훈련되고, 전체 과정을 반복하다 지정된 예측 모델의 수에 도달하거나, 완벽한 예측 모델이 만들어지면 중지된다.
- 에이다부스트의 예측은 모든 예측 모델의 예측을 계산하고, 예측 모델의 가중치 $\alpha_j$를 더해 예측 결과를 만든다. 가중치 합이 가장 큰 클래스가 예측 결과가 된다.
- 사이킷런은 SAMME라는 에이다부스트의 다중 클래스 버전을 사용한다. 클래스가 두 개이면 에이다부스트와 동일하다.
  + 예측 모델에 `predict_proba()` 메서드가 있다면 SAMME.R을 사용하여 예측값 대신 클래스 확률에 기반하여 성능이 더 좋은 모델을 만들 수 있다.
- 다음 코드는 `AdaBoostClassifier`를 사용하여 200개의 아주 얕은 결정 트리를 기반으로 하는 에이다 부스트 분류 모델 훈련 과정이다.

In [19]:
from sklearn.ensemble import AdaBoostClassifier

ada_clf = AdaBoostClassifier(
    DecisionTreeClassifier(max_depth=1), n_estimators=200,
    algorithm="SAMME.R", learning_rate=0.5, random_state=42)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1),
                   learning_rate=0.5, n_estimators=200, random_state=42)

#### | **7.5.2 그레디언트 부스팅**
- **그레디언트 부스팅(gradient boosting)** 역시 에이다부스트처럼 앙상블에 이전까지의 오차를 보정하도록 예측 모델을 순차적으로 추가한다. 하지만 에이다부스트처럼 반복마다 샘플의 가중치를 수정하는 대신 이전 예측기가 만든 잔차(residual error)에 새로운 예측 모델을 학습시킨다.
- 결정 트리 기반인 예측 모델을 사용하여 간단한 회귀 문제를 풀어보자. 이를 그레디언트 트리 부스팅 혹은 그레디언트 부스티드 회귀 트리라고 한다. 먼저 `DecisionTreeRegressor`를 훈련 세트에 학습시켜보자.

In [20]:
# data
np.random.seed(42)
X = np.random.rand(100, 1) - 0.5
y = 3*X[:, 0]**2 + 0.05 * np.random.randn(100)

In [21]:
# model fitting : DecisionTreeRegressor
from sklearn.tree import DecisionTreeRegressor

tree_reg1 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg1.fit(X, y)

DecisionTreeRegressor(max_depth=2, random_state=42)

- 이제 첫번째 예측 모델에서 생긴 잔차에 대해 두 번째 `DecisionTreeRegressor`를 훈련시킨다.

In [22]:
y2 = y - tree_reg1.predict(X) # 잔차

# 두 번째 DecisionTreeRegressor 학습
tree_reg2 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg2.fit(X, y2)

DecisionTreeRegressor(max_depth=2, random_state=42)

- 그런 다음 두 번째 예측 모델이 만든 잔차에 세 번째 회귀 모델을 훈련시킨다.

In [23]:
y3 = y - tree_reg2.predict(X) # 잔차

# 세 번째 DecisionTreeRegressor 학습
tree_reg3 = DecisionTreeRegressor(max_depth=2, random_state=42)
tree_reg3.fit(X, y3)

DecisionTreeRegressor(max_depth=2, random_state=42)

- 이제 세 개의 트리를 포함하는 앙상블 모델이 생겼다. 새로운 샘플에 대한 예측을 만들려면 모든 트리의 예측을 더하면 된다.

In [24]:
# 테스트셋 생성
X_new = np.array([[0.8]])

In [25]:
y_pred = sum(tree.predict(X_new) for tree in (tree_reg1, tree_reg2, tree_reg3))

- 사이킷런의 `GradientBoostingRegressor`를 사용하면 GBRT 앙상블을 간단하게 훈련시킬 수 있다.

In [26]:
from sklearn.ensemble import GradientBoostingRegressor

gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=3, learning_rate=1.0, random_state=42)
gbrt.fit(X, y)

GradientBoostingRegressor(learning_rate=1.0, max_depth=2, n_estimators=3,
                          random_state=42)

- `learning_rate` 매개변수가 각 트리의 기여도를 조절한다. 위처럼 0.1로 낮게 설정하면 학습시키기 위해 많은 트리가 필요하지만 예측 성능은 좋아진다. 이 규제 방법을 축소(shrinkage)라고 한다.
- 훈련 세트에 과대적합되는 것을 막는 최적의 트리 수를 찾기 위해서는 조기 종료 기법을 사용할 수 있다. 간단하게 `staged_predict(0` 메서드를 사용해 구현할 수 있다.

In [27]:
# import package and split data
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

X_train, X_val, y_train, y_val = train_test_split(X, y, random_state=49)

In [28]:
gbrt = GradientBoostingRegressor(max_depth=2, n_estimators=120, random_state=42)
gbrt.fit(X_train, y_train)

# 조기 종류 기법으로 트리 수 튜닝
errors = [mean_squared_error(y_val, y_pred)
          for y_pred in gbrt.staged_predict(X_val)]
bst_n_estimators = np.argmin(errors) + 1

In [29]:
# 최적 모델 학습
gbrt_best = GradientBoostingRegressor(max_depth=2, n_estimators=bst_n_estimators, random_state=42)
gbrt_best.fit(X_train, y_train)

GradientBoostingRegressor(max_depth=2, n_estimators=56, random_state=42)

- 실제로 훈련을 중지하는 방법으로 조기 종료를 구현할 수도 있다. `warm_start=True`로 설정하면, 사이킷런이 `fit()` 메서드가 호출될 때 기존 트리를 유지하면서 훈련을 추가한다.
- 다음 코드는 연속해서 다섯 번의 반복 동안 검증 오차가 향상되지 않으면 훈련을 멈춘다.

In [30]:
gbrt = GradientBoostingRegressor(max_depth=2, warm_start=True, random_state=42) # 기존 트리를 유지하면서 훈련을 추가

min_val_error = float("inf")
error_going_up = 0

for n_estimators in range(1, 120):
    gbrt.n_estimators = n_estimators

    # model fitting and test
    gbrt.fit(X_train, y_train)
    y_pred = gbrt.predict(X_val)
    val_error = mean_squared_error(y_val, y_pred)

    # early stoping
    if val_error < min_val_error:
        min_val_error = val_error # 검증 오차 업데이트
        error_going_up = 0
    else:
        error_going_up += 1 # 검증 오차가 감소하지 않는 횟수 +1
        if error_going_up == 5:
            break  # early stopping

- `GradientBoostingRegressor`는 각 트리가 훈련할 때 사용할 훈련 샘플의 비율을 지정하는 `subsample` 매개 변수를 지원한다. 이를 이용한 방법을 확률적 그레디언트 부스팅이라고 한다.
- XGBoost 파이썬 라이브러리를 이용하여 그레디언트 부스팅을 구현할 수도 있다.

In [31]:
import xgboost

xgb_reg = xgboost.XGBRegressor(random_state=42)
xgb_reg.fit(X_train, y_train)
y_pred = xgb_reg.predict(X_val)

[21:06:17] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [32]:
# 자동 조기 종료 기능
xgb_reg.fit(X_train, y_train, eval_set=[(X_val, y_val)], early_stopping_rounds=2)
y_pred = xgb_reg.predict(X_val)

[21:06:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-rmse:0.286719
Will train until validation_0-rmse hasn't improved in 2 rounds.
[1]	validation_0-rmse:0.258221
[2]	validation_0-rmse:0.232634
[3]	validation_0-rmse:0.210526
[4]	validation_0-rmse:0.190232
[5]	validation_0-rmse:0.172196
[6]	validation_0-rmse:0.156394
[7]	validation_0-rmse:0.142241
[8]	validation_0-rmse:0.129789
[9]	validation_0-rmse:0.118752
[10]	validation_0-rmse:0.108388
[11]	validation_0-rmse:0.100155
[12]	validation_0-rmse:0.09208
[13]	validation_0-rmse:0.084791
[14]	validation_0-rmse:0.078699
[15]	validation_0-rmse:0.073248
[16]	validation_0-rmse:0.069391
[17]	validation_0-rmse:0.066277
[18]	validation_0-rmse:0.063458
[19]	validation_0-rmse:0.060326
[20]	validation_0-rmse:0.0578
[21]	validation_0-rmse:0.055643
[22]	validation_0-rmse:0.053943
[23]	validation_0-rmse:0.053138
[24]	validation_0-rmse:0.052415
[25]	validation_0-rmse

## **7.6 스태킹**
____
- 스태킹(stacking)은 앙상블에 속한 모든 예측 모델의 예측을 취합하는 모델이다.
- **메타 학습기(meta learner)** 혹은 블렌더(blender)는 각 모델의 예측을 입력 받아 최종 예측을 한다.
- 블렌더를 학습시키는 일반적인 방법은 홀드 아웃(hold-out) 세트를 이용하는 것이다.
  + 먼저 훈련 세트를 두 개의 서브셋으로 나눈다.
  + 첫 번째 서브셋을 이용하여 예측 모델을 훈련시킨다.
  + 훈련된 예측 모델로 두 번째 (홀드 아웃) 셋에 대한 예측을 만든다.
  + 타겟값은 그대로 사용하고, 홀드 아웃 세트의 각 샘플에 대한 예측값들을 입력 피처로 사용하여 새로운 훈련 세트를 만든다.
  + 블렌더를 새 훈련 세트로 학습시킨다.